In [1]:
import torch.nn as nn

class SentimentAnalysisModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, 
                 output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__();
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx);
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, 
                           num_layers = n_layers, 
                           bidirectional = bidirectional,
                           dropout = dropout);
        self.fullyconnected = nn.Linear(hidden_dim*2, output_dim);
        self.dropout = nn.Dropout(dropout);
#         self.sigmoid = nn.Sigmoid()
        
    def forward(self, tweets, text_lengths):
        embedded = self.embedding(tweets);
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths);
        _,(hidden, _) = self.rnn(packed_embedded);
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1));
#         return self.sigmoid(self.fullyconnected(hidden));
        return self.fullyconnected(hidden);

In [5]:
import torch
import os.path
import pickle

def read_vocab(path):
    pkl_file = open(path, 'rb')
    vocab = pickle.load(pkl_file)
    pkl_file.close()
    return vocab

def load_model_and_vocab(vocab_path, weights_path):    
    vocab = read_vocab(vocab_path)
    model = SentimentAnalysisModel(len(vocab),
                              embedding_dim = 100,
                              hidden_dim = 256,
                              output_dim = 1,
                              n_layers = 2,
                              bidirectional = True,
                              dropout = 0.5,
                              pad_idx = 1);
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    return model, vocab

model, vocab = load_model_and_vocab("sentiment_vocab.pkl", "sentiment_analysis_weights.pt");

In [34]:
import spacy
import re
nlp = spacy.load("en_core_web_sm");

def clean_tweets(text):
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text);
    text = re.sub(r'https?:/\/\S+', ' ', text);
    return text.strip();

def tokenizer(text):
    return [w.text.lower() for w in nlp(clean_tweets(text))];

def sentence_to_input(text):
    tokens = tokenizer(text);
    length = len(tokens);
    input_vector = [vocab[x] for x in tokens];
    input_vector = torch.LongTensor(input_vector);
    return (torch.unsqueeze(input_vector, 1), torch.LongTensor([length]));

tweets = ['I love the way Elon Musk is expressing his monopoly over twitter.', 
          'Your act was not good.']
for tweet in tweets:
    text, text_lengths = sentence_to_input(tweet);
    pred = model(text, text_lengths).squeeze();
    rounded_pred = torch.round(torch.sigmoid(pred));
    labels = ['pos', 'neg'];
    print(tweet, ' : ', labels[int(rounded_pred.item())]);

I love the way Elon Musk is expressing his monopoly over twitter.  :  pos
Your act was not good.  :  neg
